In [ ]:
%pip install bs4 --quiet
%pip install pandas --quiet
%pip install lxml --quiet
%pip install PyPDF2 --quiet
%pip install openai==0.28.1 --quiet 
%pip install langchain==0.0.320 --quiet

In [ ]:
import requests
from PyPDF2 import PdfReader
from io import BytesIO
from bs4 import BeautifulSoup
import pandas as pd

Egyetemi szabályzat betöltése a pdf formátumból

In [ ]:
pdf_url = 'https://www.vik.bme.hu/document/412/original/Szakmai_gyak_BSc_szabályzat_2014 után.pdf'
response = requests.get(pdf_url)
pdf_content = response.content

if response.status_code == 200:
    pdf_bytes = response.content
    pdf_file = BytesIO(pdf_bytes)

    pdf_reader = PdfReader(pdf_file)
    num_pages = len(pdf_reader.pages)

    # Az oldalak szövegének kinyerése és összefűzése
    text = ""
    for page in range(num_pages):
        page_obj = pdf_reader.pages[page]
        text += page_obj.extract_text()


    with open('res/3.txt', 'w', encoding='utf-8') as f:
        f.write(text)

    pdf_file.close()
else:
    print("Failed to download the PDF file.")

AUT-os GYIK betöltése dataframe-be

In [ ]:
url = 'https://www.aut.bme.hu/SzakmaiGyakorlat/GyIK'
qna = []

response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')
questions = soup.find_all(class_='question')
for q in questions:
    qna.append([q.get_text().strip(), q.find_next_sibling().get_text().strip()])

qna

Dataframe kimentése csv fájlba

In [ ]:
# Listából dataframe-et készítünk
df = pd.DataFrame(qna, columns=['Question', 'Answer'])

# dataframe-et csv-be mentjük
df.to_csv('res/gyik.csv', index= False)

3 gyakorlatinfókkal teli oldal tartalmának kinyerése, ebből az egyik egy táblázat, ami dataframe-be kerül és úgy mentve az 1.txt-be

In [ ]:
urls = ['https://www.aut.bme.hu/SzakmaiGyakorlat/', 'https://www.aut.bme.hu/SzakmaiGyakorlat/Teendok', 'https://www.vik.bme.hu/kepzes/gyakorlat/442.html']

for url in urls:
    print(url)
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    #az oldalakon itt van a fő tartalom eltárolva
    def has_role_main_or_id_main(tag):
        return (tag.get('role') == 'main') or (tag.get('id') == 'main')

    content = soup.find_all(has_role_main_or_id_main)

    # A Teendok oldal egy táblázatot fog tartalmazni, ezt külön kell kezelni, akár külön cella is lehetne annak a kezelése
    tables = soup.find_all('table')

    
    if (urls.index(url) != 1):
        with open(f'res/{urls.index(url)}.txt', 'w', encoding='utf-8') as f:
            for elem in content:
                text = elem.get_text()
                # Sorokra bontjuk a szöveget, eltávolítjuk a felesleges whitespace-eket, és kihagyjuk az üres sorokat
                lines = [line.strip() for line in text.splitlines() if line.strip()]
                # Ezek eután újra összraj a sorokat
                text = '\n'.join(lines)
                f.write(text)
    else:
        # Az egyes indexű oldal egy táblázatot fog tartalmazni, ezt csv-be mentjük
        for table in enumerate(tables):
            todo_df = pd.read_html(str(table))[0]

            todo_df.to_csv(f'./res/{urls.index(url)}.csv', index=False)

## Kérdések generálása a többi oldalhoz is

In [ ]:
# Resource-ok betöltése, nem kell újra az oldalakat scrapelni
from langchain.document_loaders import DirectoryLoader, TextLoader

directory_loader = DirectoryLoader("res", glob="*/*.txt", use_multithreading=True, loader_cls=TextLoader, loader_kwargs={"encoding": "utf-8"})
text_data = directory_loader.load()
text_data

In [ ]:
from langchain.text_splitter import TokenTextSplitter

# A szövegeket 250 karakteres chunk-okra bontjuk, amiknek 50 karakteres az átfedése
splitter = TokenTextSplitter(chunk_size=250, chunk_overlap=50)

splitted_data = splitter.split_documents(text_data)
print(len(splitted_data))

In [ ]:
import openai
import os
import pandas as pd
openai.api_key = os.getenv("OPENAI_API_KEY")

In [1]:
# Egy szövegrészlethez kérdések generálása
def generate_faq_questions_answers(text):    
    response = openai.ChatCompletion.create(
        model='gpt-3.5-turbo',
        messages=[
            {"role": "system", "content": "You are a FAQ question and answer generator."},
            {"role": "user", "content": f"""
             Generate 15 FAQ-like question-answer pairs for the given text. If you don't know the answer, don't make up an answer and  don't give me the question .
             Please provide your answer in Hungarian. Enter the question and answer separated by a ; character.
             
             Text:
             {text}
             
             Example:
             "Mi a feladata a hallgatónak?";"A szakmai gyakorlati helyen a szakmai gyakorlati feladatokat elvégezze, és azt igazolja, illetve beszámolót készítsen hozzá."
             "Mikor kell felvenni a szakmai gyakorlat tárgyat?";"A szakmai gyakorlat félévében, nyári gyakorlat esetén a következő őszi félévben."
             """
            }
        ],
        temperature=0.75,
        #max_tokens=500,
       )

    return response

In [ ]:
generated_df = pd.DataFrame(columns=['Question', 'Answer'])

for part in splitted_data:
    prompt = part.page_content
    response = generate_faq_questions_answers(prompt)
    lines = response['choices'][0]['message']['content'].split('\n')

    data = []
    # A válaszokat ; karakter mentén szétválasztjuk, és a kérdést és a választ külön-külön eltároljuk
    for line in lines:
        qna = line.split(';')
        question = qna[0].strip('"')
        answer = qna[1].strip('"')
        data.append([question, answer])

    # A generált kérdéseket és válaszokat egy dataframe-be mentjük
    generated_df = pd.concat([generated_df, pd.DataFrame(data, columns=['Question','Answer'])], ignore_index=True)

In [ ]:
generated_df.to_csv(f'res/generated_qna{splitted_data.index(part)}.csv', index=False)

#### Generate with Langchain (only in English)

In [ ]:
from langchain.evaluation.qa import QAGenerateChain
from langchain.chat_models import ChatOpenAI

model = ChatOpenAI(model= "gpt-3.5-turbo")
gen_chain = QAGenerateChain.from_llm(model)

In [ ]:
questions_and_answers = gen_chain.apply_and_parse([{"doc": t} for t in splitted_data])

In [ ]:
import json
with open('res/qna.json', 'w') as f:
    json.dump(questions_and_answers, f, indent=4)

## csv delimiter kompatibilitás miatti rész (Excel ; delimiterrel dolgozik, ezért a review után ezzel mentette le)

In [ ]:
gpt35_df = pd.read_csv('res/not_in_use/generated_qna_GPT3.5_reviewed_excel.csv', delimiter=';')
gpt35_df.to_csv('res/in_use/generated_qna_GPT3.5_reviewed.csv', index=False)

gpt4_df = pd.read_csv('res/not_in_use/generated_qna_GPT4_reviewed_excel.csv', delimiter=';')
gpt4_df.to_csv('res/in_use/generated_qna_GPT4_reviewed.csv', index=False)

## Blázovics Lacitól kapott mailekből személyes infók kivágása

In [ ]:
import pandas as pd

# a kapott emailek beolvasása
email_df = pd.read_csv('res/secret/SzakGyakQnA_original.csv', header=None)

In [3]:
# Milye szavakat tartalmazó sorokat kell kihagyni
remove_words = ['From:', 'To:', 'Cc:', 'Sent:', 'Date:', 'mailto',
                'Feladó:', 'Küldve:', 'Címzett:', 'Másolatot kap:', 'Dátum:', 'Címe:',
                'Kedves', 'Tisztelt',
                'Üdvözlettel', 'Üdv', 'Tisztelettel', 'Köszönettel', 'Köszönöm', 'Válaszát', 'A választ'
                'Neptun kód', 'Neptun:', 'Vírusmentes', 'Iroda:', 'Cím:', 'Mobil:', 'Cégbíróság:', 'Cg', 'Adószám:', 'Bankszámlaszám:', 'Email:' ,'Web:', '@',
                '<https://www.avast.', '<http://www.avg', '+36', '06']

In [ ]:
clean_df = pd.DataFrame(columns=['Emails'])

for index, row in email_df.iterrows():
  lines = row[0].split('\n')
  # Minden sor ellenőrzése, hogy tartalmazza-e a remove_words-ben megadott szavakat vagy üres-e vagy 6 karakter hosszú-e (talán neptunt ki lehetne szűrni így és más, fontosat nem)
  clean_lines = [line for line in lines if not any(word in line.strip() for word in remove_words) and len(line) != 6]
  # clean_lines lista összefűzése egy stringgé, majd a clean_df-hez adása
  clean_df = pd.concat([clean_df, pd.DataFrame([['\n'.join(clean_lines)]], columns=['Emails'])], ignore_index=True)

clean_df

In [ ]:
clean_df.to_csv('res/secret/SzakGyakQnA_cleaned.csv', index=False, header=None)

Kérdések generálása Openai-jal

In [ ]:
import openai
import os
import pandas as pd
openai.api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
def generate_questions_from_email(text):   
    response = openai.ChatCompletion.create(
        model='gpt-3.5-turbo',
        messages=[
            {"role": "system", "content": "You are a FAQ question and answer generator."},
            {"role": "user", "content": f"""
             Generate a few FAQ-like question-answer pairs for the given text. The text is an email conversation stripped from most of the personal info. The latest email comes first, then the previous one, and so on. The last email was originally the first.
             Please provide your answer in Hungarian. Enter the question and answer separated by a ; character as in a csv file.
             
             Email conversation:
             {text}
             
             Example:
             "Mi a feladata a hallgatónak?";"A szakmai gyakorlati helyen a szakmai gyakorlati feladatokat elvégezze, és azt igazolja, illetve beszámolót készítsen hozzá."
             "Mikor kell felvenni a szakmai gyakorlat tárgyat?";"A szakmai gyakorlat félévében, nyári gyakorlat esetén a következő őszi félévben."
             """
            }
        ],
        temperature=0.75,
        #max_tokens=500,
       )

    return response

In [ ]:
import time

email_gen_df = pd.DataFrame(columns=['Question', 'Answer'])

for index, row in clean_df.iterrows():
    prompt = row['Emails']
    response = generate_questions_from_email(prompt)
    lines = response['choices'][0]['message']['content'].split('\n')

    data = []
    for line in lines:
        try:
            qna = line.split(';')
            question = qna[0].strip('"')
            answer = qna[1].strip('"')
            data.append([question, answer])
        except Exception as e:
           print(f"Error processing line: {line}")
           print(f"Error message: {str(e)}")

    email_gen_df = pd.concat([email_gen_df, pd.DataFrame(data, columns=['Question','Answer'])], ignore_index=True)
    time.sleep(1)
email_gen_df.to_csv(f'res/generated_email_qna.csv', index=False)

In [ ]:
email_gen_df

In [ ]:
## Reviewed csv tagolása rossz itt is Excel miatt
gen_email_rev_df = pd.read_csv('res/in_use/generated_email_qna_reviewed.csv', delimiter=';')
gen_email_rev_df.to_csv('res/in_use/generated_email_qna_reviewed.csv', index=False)